In [3]:
!pip install deepctr

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras_preprocessing.sequence import pad_sequences
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, get_feature_names
from deepctr.models.xdeepfm import xDeepFM
from sklearn.model_selection import train_test_split

In [5]:
# user log data import
search = pd.read_csv('search.csv', encoding='UTF-8')
detail_title = pd.read_csv('detail_title.csv', encoding='UTF-8')
wish_title = pd.read_csv('wish_title.csv', encoding='UTF-8')

In [6]:
category_vector = pd.read_csv('category_vector.csv') # doc2vec embedding
vector = pd.read_csv("vectors.csv", encoding = 'utf8') # 키워드 vector
bert_df = pd.read_csv("isbn_title.csv", encoding = 'cp949')# all data import
all_book = pd.read_csv('한얼님께받은전체데이터.csv')

# 전체 data 생성

## author publisher 분리된column얻기
## catego"rydata에 없으므로 merge해주어야함

In [6]:
def date_encoding(x):
    return x.split('-')[0]

In [7]:
category_vector['date'] = category_vector['date'].apply(date_encoding) # 날짜 전처리(년도만 분리)

In [8]:
all_book.columns

Index(['id', 'isbn', 'kwd', 'title', 'date', 'Cn_1', 'Cn_2', 'Cn_3', 'Cn_4',
       'author', 'publisher', 'img_url'],
      dtype='object')

In [9]:
all_book.drop(columns = ['id','kwd','Cn_1','Cn_2','Cn_3','Cn_4','img_url','date'],inplace=True)

In [10]:
all_book2 = pd.merge(left = category_vector , right =all_book, how = "left", on = "isbn")

In [11]:
all_book2.drop_duplicates(['isbn'],inplace=True)

In [12]:
bertData = pd.concat([bert_df,vector],axis = 1)
kwd_bert = pd.merge(left = bertData , right = all_book2, how = "left", on = "isbn")
kwd_bert # 책들의 키워드 벡터

,isbn,title,0,1,2,3,4,5,6,7,...,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,vector,title_y,author,publisher
0,2000151036468,성경(46판 2단 금장색인) (BB6 BH) (250130),0.889670,-1.926874,-1.562464,0.118039,2.472656,1.143712,-3.656446,-1.393214,...,2006,종교,가톨릭,가톨릭성서,가톨릭성서,가톨릭출판사 편집부가톨릭출판사,"[-0.19663704931735992, 0.02086380310356617, 0....",성경(46판 2단 금장색인) (BB6 BH) (250130),가톨릭출판사 편집부,가톨릭출판사
1,2000127000059,남북전육부 율장비교연구(3판),0.100628,1.642053,0.184597,-0.355000,4.472329,1.346205,-0.573288,-1.483691,...,1976,종교,불교,불교철학사상,불교철학사상,이지관가산문고,"[-0.19683720171451569, -0.1224818155169487, 0....",남북전육부 율장비교연구(3판),이지관,가산문고
2,2000107000024,제7감각을 기르자,-0.652296,-0.094557,0.932296,0.701581,0.659830,-0.379887,-2.638436,0.178885,...,1994,인문,교육학,교수학습법,학습법,죤 니스벳 자넷 셕스미쓰원미사,"[-0.13387304544448853, -0.0697629377245903, 0....",제7감각을 기르자,"죤 니스벳, 자넷 셕스미쓰",원미사
3,2000156000150,디오니소스의 노동 1,0.208688,-0.472835,-1.438581,0.343109,-0.204737,1.738672,0.094855,2.180434,...,1996,정치사회,사회문제복지,노동문제,노동문제일반,안또니오 네그리갈무리,"[-0.07980803400278091, 0.12560898065567017, -0...",디오니소스의 노동 1,안또니오 네그리,갈무리
4,2000156000198,아침햇살이 그립다,-0.151617,-0.381259,0.429895,0.386737,0.872832,-0.665152,-1.574062,0.657493,...,2001,시에세이,한국시,현대시,현대시,표성배갈무리,"[0.17261381447315216, -0.07646216452121735, 0....",아침햇살이 그립다,표성배,갈무리
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026556,3902000200183,(사은품)파고다 고득점 비법서,-0.294171,-0.753573,-2.994084,0.327008,-0.174073,-2.105743,-1.116584,-1.220681,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1026557,3902000200190,(사은품)(파고다)Grammar Chat 복습북,-1.382363,-3.426793,-2.555956,-0.204006,1.022561,-0.699903,-1.718663,-0.356797,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1026558,3902000200206,(사은품)[웅진주니어] 도토리 판퍼즐 ver.1,0.814329,-0.571693,-1.849404,0.003993,1.212836,0.277659,-0.898165,2.594340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1026559,3902000200213,(사은품)[웅진주니어] 도토리 판퍼즐 ver.2,0.769123,-0.612501,-1.877810,0.002436,1.294599,0.273753,-0.930771,2.536193,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
kwd_bert.drop_duplicates(['isbn'],inplace=True)

In [14]:
kwd_bert.dropna(inplace=True)

In [15]:
kwd_bert

,isbn,title,0,1,2,3,4,5,6,7,...,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,vector,title_y,author,publisher
0,2000151036468,성경(46판 2단 금장색인) (BB6 BH) (250130),0.889670,-1.926874,-1.562464,0.118039,2.472656,1.143712,-3.656446,-1.393214,...,2006,종교,가톨릭,가톨릭성서,가톨릭성서,가톨릭출판사 편집부가톨릭출판사,"[-0.19663704931735992, 0.02086380310356617, 0....",성경(46판 2단 금장색인) (BB6 BH) (250130),가톨릭출판사 편집부,가톨릭출판사
1,2000127000059,남북전육부 율장비교연구(3판),0.100628,1.642053,0.184597,-0.355000,4.472329,1.346205,-0.573288,-1.483691,...,1976,종교,불교,불교철학사상,불교철학사상,이지관가산문고,"[-0.19683720171451569, -0.1224818155169487, 0....",남북전육부 율장비교연구(3판),이지관,가산문고
2,2000107000024,제7감각을 기르자,-0.652296,-0.094557,0.932296,0.701581,0.659830,-0.379887,-2.638436,0.178885,...,1994,인문,교육학,교수학습법,학습법,죤 니스벳 자넷 셕스미쓰원미사,"[-0.13387304544448853, -0.0697629377245903, 0....",제7감각을 기르자,"죤 니스벳, 자넷 셕스미쓰",원미사
3,2000156000150,디오니소스의 노동 1,0.208688,-0.472835,-1.438581,0.343109,-0.204737,1.738672,0.094855,2.180434,...,1996,정치사회,사회문제복지,노동문제,노동문제일반,안또니오 네그리갈무리,"[-0.07980803400278091, 0.12560898065567017, -0...",디오니소스의 노동 1,안또니오 네그리,갈무리
4,2000156000198,아침햇살이 그립다,-0.151617,-0.381259,0.429895,0.386737,0.872832,-0.665152,-1.574062,0.657493,...,2001,시에세이,한국시,현대시,현대시,표성배갈무리,"[0.17261381447315216, -0.07646216452121735, 0....",아침햇살이 그립다,표성배,갈무리
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026524,9791186762196,산꾼 정성완의 산이야기,-1.734667,-0.330009,-1.643743,2.104612,-0.118947,1.265014,-2.636759,-1.365183,...,2022,여행,여행,여행,여행,정성완시선,"[-0.05782816931605339, 0.06711617112159729, -0...",산꾼 정성완의 산이야기,정성완,시선
1026525,2090000081992,종가제례음식 경상편(종가제례음식총서 5)(양장본 HardCover),-0.386263,2.181529,-3.371171,0.852583,0.905377,-0.055213,-1.563818,-1.766419,...,2020,역사문화,민속학,가정의례,가정의례,한국국학진흥원 기획민속원,"[-0.16335245966911316, -0.0906568393111229, 0....",종가제례음식 경상편(종가제례음식총서 5)(양장본 HardCover),한국국학진흥원 (기획),민속원
1026536,9788974402747,가이디드타로,0.555394,-2.088872,0.291375,1.224848,-0.588969,-0.327170,2.203317,-0.540690,...,2022,취미실용스포츠,취미실용스포츠,취미실용스포츠,취미실용스포츠,스테파니 카포니일월서각,"[-0.17115232348442078, 0.0751611664891243, 0.3...",가이디드타로,스테파니 카포니,일월서각
1026537,9791197704208,현대 일본 정치의 이해,0.296418,-0.459678,-1.140749,0.784226,2.133747,2.150167,-2.498011,-2.850572,...,2022,정치사회,정치사회,정치사회,정치사회,홍성창에스씨에이치디,"[-0.1670745313167572, 0.10448136180639267, -0....",현대 일본 정치의 이해,홍성창,에스씨에이치디


In [17]:
def strtolist(x):
    return eval(x)

In [19]:
kwd_bert['vector'] = kwd_bert['vector'].apply(strtolist)

In [20]:
vector_df = pd.DataFrame(kwd_bert['vector'].values.tolist()).add_prefix('catev_').join(kwd_bert)

In [21]:
vector_df

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,vector,title_y,author,publisher
0,-0.196637,0.020864,0.350972,0.192017,-0.206795,-0.054141,0.136589,-0.118211,-0.104625,-0.071694,...,2006,종교,가톨릭,가톨릭성서,가톨릭성서,가톨릭출판사 편집부가톨릭출판사,"[-0.19663704931735992, 0.02086380310356617, 0....",성경(46판 2단 금장색인) (BB6 BH) (250130),가톨릭출판사 편집부,가톨릭출판사
1,-0.196837,-0.122482,0.348215,0.117282,-0.099319,-0.070873,0.216196,0.005626,-0.180764,-0.030805,...,1976,종교,불교,불교철학사상,불교철학사상,이지관가산문고,"[-0.19683720171451569, -0.1224818155169487, 0....",남북전육부 율장비교연구(3판),이지관,가산문고
2,-0.133873,-0.069763,0.103495,0.018553,-0.203616,-0.056440,0.215832,0.082005,-0.384968,-0.267825,...,1994,인문,교육학,교수학습법,학습법,죤 니스벳 자넷 셕스미쓰원미사,"[-0.13387304544448853, -0.0697629377245903, 0....",제7감각을 기르자,"죤 니스벳, 자넷 셕스미쓰",원미사
3,-0.079808,0.125609,-0.159979,-0.060015,0.081533,-0.066703,0.242454,0.133134,-0.482992,0.177329,...,1996,정치사회,사회문제복지,노동문제,노동문제일반,안또니오 네그리갈무리,"[-0.07980803400278091, 0.12560898065567017, -0...",디오니소스의 노동 1,안또니오 네그리,갈무리
4,0.172614,-0.076462,0.229195,0.080075,0.038470,0.082090,0.004960,0.132072,-0.410798,-0.079386,...,2001,시에세이,한국시,현대시,현대시,표성배갈무리,"[0.17261381447315216, -0.07646216452121735, 0....",아침햇살이 그립다,표성배,갈무리
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000358,-0.057828,0.067116,-0.023779,0.157665,-0.401717,-0.264535,0.191748,-0.234187,-0.223327,0.101256,...,2008,만화,로맨스만화,로맨스만화,로맨스만화,신일숙학산문화사,"[-0.22375155985355377, -0.09855737537145615, 0...",아르미안의 네 딸들 10(신일숙 환상전집 1),신일숙,학산문화사
1000359,-0.163352,-0.090657,0.200129,0.268317,-0.316147,0.228996,0.366973,-0.162490,-0.195312,0.101570,...,2009,만화,로맨스만화,로맨스만화,로맨스만화,한유랑삼양출판사,"[-0.025331787765026093, 0.04532694071531296, 0...",너에게 간다 9(완결),한유랑,삼양출판사
1000360,-0.171152,0.075161,0.338542,0.336318,-0.265556,-0.217217,0.132284,0.056170,-0.285489,0.117279,...,2008,요리,와인커피음료,와인,와인,이동현김정,"[-0.03224857151508331, 0.30024009943008423, 0....",즐거운 와인 여행 2(포켓북(문고판)),이동현,김&정
1000361,-0.167075,0.104481,-0.072516,0.121800,-0.037745,-0.094742,0.241081,0.018632,-0.512108,0.105987,...,2008,만화,그래픽노블,판타지,판타지,애니북스,"[-0.1490665078163147, 0.04922980070114136, 0.3...",스타워즈 STAR WARS 클론워즈 어드벤처 5,DARK HORSE BOOKS,애니북스


In [22]:
vector_df.drop(columns = ['title_y','vector'],inplace=True)

In [23]:
vector_df

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,kwd,title_x,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher
0,-0.196637,0.020864,0.350972,0.192017,-0.206795,-0.054141,0.136589,-0.118211,-0.104625,-0.071694,...,성경 교양서 가톨릭 금장 이해 색인,성경(46판 2단 금장색인) (BB6 BH) (250130),2006,종교,가톨릭,가톨릭성서,가톨릭성서,가톨릭출판사 편집부가톨릭출판사,가톨릭출판사 편집부,가톨릭출판사
1,-0.196837,-0.122482,0.348215,0.117282,-0.099319,-0.070873,0.216196,0.005626,-0.180764,-0.030805,...,교양서 불교 남북 연구 이해 율장 비교,남북전육부 율장비교연구(3판),1976,종교,불교,불교철학사상,불교철학사상,이지관가산문고,이지관,가산문고
2,-0.133873,-0.069763,0.103495,0.018553,-0.203616,-0.056440,0.215832,0.082005,-0.384968,-0.267825,...,감각,제7감각을 기르자,1994,인문,교육학,교수학습법,학습법,죤 니스벳 자넷 셕스미쓰원미사,"죤 니스벳, 자넷 셕스미쓰",원미사
3,-0.079808,0.125609,-0.159979,-0.060015,0.081533,-0.066703,0.242454,0.133134,-0.482992,0.177329,...,노동 디오니소스 교양 전공 전문 이해 대상,디오니소스의 노동 1,1996,정치사회,사회문제복지,노동문제,노동문제일반,안또니오 네그리갈무리,안또니오 네그리,갈무리
4,0.172614,-0.076462,0.229195,0.080075,0.038470,0.082090,0.004960,0.132072,-0.410798,-0.079386,...,아침 햇살,아침햇살이 그립다,2001,시에세이,한국시,현대시,현대시,표성배갈무리,표성배,갈무리
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000358,-0.057828,0.067116,-0.023779,0.157665,-0.401717,-0.264535,0.191748,-0.234187,-0.223327,0.101256,...,신일숙 아르 미안 전집 환상,아르미안의 네 딸들 10(신일숙 환상전집 1),2008,만화,로맨스만화,로맨스만화,로맨스만화,신일숙학산문화사,신일숙,학산문화사
1000359,-0.163352,-0.090657,0.200129,0.268317,-0.316147,0.228996,0.366973,-0.162490,-0.195312,0.101570,...,완결,너에게 간다 9(완결),2009,만화,로맨스만화,로맨스만화,로맨스만화,한유랑삼양출판사,한유랑,삼양출판사
1000360,-0.171152,0.075161,0.338542,0.336318,-0.265556,-0.217217,0.132284,0.056170,-0.285489,0.117279,...,와인 포켓북 문고판 여행,즐거운 와인 여행 2(포켓북(문고판)),2008,요리,와인커피음료,와인,와인,이동현김정,이동현,김&정
1000361,-0.167075,0.104481,-0.072516,0.121800,-0.037745,-0.094742,0.241081,0.018632,-0.512108,0.105987,...,작품 소설 스토리 단행본 만화 에피소드 클론 영화 시리즈 애니메이션,스타워즈 STAR WARS 클론워즈 어드벤처 5,2008,만화,그래픽노블,판타지,판타지,애니북스,DARK HORSE BOOKS,애니북스


In [24]:
vector_df.columns

Index(['catev_0', 'catev_1', 'catev_2', 'catev_3', 'catev_4', 'catev_5',
       'catev_6', 'catev_7', 'catev_8', 'catev_9',
       ...
       'kwd', 'title_x', 'date', 'Cn_1', 'Cn_2', 'Cn_3', 'Cn_4',
       'author_publisher', 'author', 'publisher'],
      dtype='object', length=348)

In [26]:
vector_df.rename(columns={'title_x':'title'},inplace=True)

In [30]:
vector_df

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,kwd,title,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher
0,-0.196637,0.020864,0.350972,0.192017,-0.206795,-0.054141,0.136589,-0.118211,-0.104625,-0.071694,...,성경 교양서 가톨릭 금장 이해 색인,성경(46판 2단 금장색인) (BB6 BH) (250130),2006,종교,가톨릭,가톨릭성서,가톨릭성서,가톨릭출판사 편집부가톨릭출판사,가톨릭출판사 편집부,가톨릭출판사
1,-0.196837,-0.122482,0.348215,0.117282,-0.099319,-0.070873,0.216196,0.005626,-0.180764,-0.030805,...,교양서 불교 남북 연구 이해 율장 비교,남북전육부 율장비교연구(3판),1976,종교,불교,불교철학사상,불교철학사상,이지관가산문고,이지관,가산문고
2,-0.133873,-0.069763,0.103495,0.018553,-0.203616,-0.056440,0.215832,0.082005,-0.384968,-0.267825,...,감각,제7감각을 기르자,1994,인문,교육학,교수학습법,학습법,죤 니스벳 자넷 셕스미쓰원미사,"죤 니스벳, 자넷 셕스미쓰",원미사
3,-0.079808,0.125609,-0.159979,-0.060015,0.081533,-0.066703,0.242454,0.133134,-0.482992,0.177329,...,노동 디오니소스 교양 전공 전문 이해 대상,디오니소스의 노동 1,1996,정치사회,사회문제복지,노동문제,노동문제일반,안또니오 네그리갈무리,안또니오 네그리,갈무리
4,0.172614,-0.076462,0.229195,0.080075,0.038470,0.082090,0.004960,0.132072,-0.410798,-0.079386,...,아침 햇살,아침햇살이 그립다,2001,시에세이,한국시,현대시,현대시,표성배갈무리,표성배,갈무리
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000358,-0.057828,0.067116,-0.023779,0.157665,-0.401717,-0.264535,0.191748,-0.234187,-0.223327,0.101256,...,신일숙 아르 미안 전집 환상,아르미안의 네 딸들 10(신일숙 환상전집 1),2008,만화,로맨스만화,로맨스만화,로맨스만화,신일숙학산문화사,신일숙,학산문화사
1000359,-0.163352,-0.090657,0.200129,0.268317,-0.316147,0.228996,0.366973,-0.162490,-0.195312,0.101570,...,완결,너에게 간다 9(완결),2009,만화,로맨스만화,로맨스만화,로맨스만화,한유랑삼양출판사,한유랑,삼양출판사
1000360,-0.171152,0.075161,0.338542,0.336318,-0.265556,-0.217217,0.132284,0.056170,-0.285489,0.117279,...,와인 포켓북 문고판 여행,즐거운 와인 여행 2(포켓북(문고판)),2008,요리,와인커피음료,와인,와인,이동현김정,이동현,김&정
1000361,-0.167075,0.104481,-0.072516,0.121800,-0.037745,-0.094742,0.241081,0.018632,-0.512108,0.105987,...,작품 소설 스토리 단행본 만화 에피소드 클론 영화 시리즈 애니메이션,스타워즈 STAR WARS 클론워즈 어드벤처 5,2008,만화,그래픽노블,판타지,판타지,애니북스,DARK HORSE BOOKS,애니북스


In [31]:
vector_df.drop_duplicates(['isbn'],inplace=True)

In [32]:
vector_df

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,kwd,title,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher
0,-0.196637,0.020864,0.350972,0.192017,-0.206795,-0.054141,0.136589,-0.118211,-0.104625,-0.071694,...,성경 교양서 가톨릭 금장 이해 색인,성경(46판 2단 금장색인) (BB6 BH) (250130),2006,종교,가톨릭,가톨릭성서,가톨릭성서,가톨릭출판사 편집부가톨릭출판사,가톨릭출판사 편집부,가톨릭출판사
1,-0.196837,-0.122482,0.348215,0.117282,-0.099319,-0.070873,0.216196,0.005626,-0.180764,-0.030805,...,교양서 불교 남북 연구 이해 율장 비교,남북전육부 율장비교연구(3판),1976,종교,불교,불교철학사상,불교철학사상,이지관가산문고,이지관,가산문고
2,-0.133873,-0.069763,0.103495,0.018553,-0.203616,-0.056440,0.215832,0.082005,-0.384968,-0.267825,...,감각,제7감각을 기르자,1994,인문,교육학,교수학습법,학습법,죤 니스벳 자넷 셕스미쓰원미사,"죤 니스벳, 자넷 셕스미쓰",원미사
3,-0.079808,0.125609,-0.159979,-0.060015,0.081533,-0.066703,0.242454,0.133134,-0.482992,0.177329,...,노동 디오니소스 교양 전공 전문 이해 대상,디오니소스의 노동 1,1996,정치사회,사회문제복지,노동문제,노동문제일반,안또니오 네그리갈무리,안또니오 네그리,갈무리
4,0.172614,-0.076462,0.229195,0.080075,0.038470,0.082090,0.004960,0.132072,-0.410798,-0.079386,...,아침 햇살,아침햇살이 그립다,2001,시에세이,한국시,현대시,현대시,표성배갈무리,표성배,갈무리
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000358,-0.057828,0.067116,-0.023779,0.157665,-0.401717,-0.264535,0.191748,-0.234187,-0.223327,0.101256,...,신일숙 아르 미안 전집 환상,아르미안의 네 딸들 10(신일숙 환상전집 1),2008,만화,로맨스만화,로맨스만화,로맨스만화,신일숙학산문화사,신일숙,학산문화사
1000359,-0.163352,-0.090657,0.200129,0.268317,-0.316147,0.228996,0.366973,-0.162490,-0.195312,0.101570,...,완결,너에게 간다 9(완결),2009,만화,로맨스만화,로맨스만화,로맨스만화,한유랑삼양출판사,한유랑,삼양출판사
1000360,-0.171152,0.075161,0.338542,0.336318,-0.265556,-0.217217,0.132284,0.056170,-0.285489,0.117279,...,와인 포켓북 문고판 여행,즐거운 와인 여행 2(포켓북(문고판)),2008,요리,와인커피음료,와인,와인,이동현김정,이동현,김&정
1000361,-0.167075,0.104481,-0.072516,0.121800,-0.037745,-0.094742,0.241081,0.018632,-0.512108,0.105987,...,작품 소설 스토리 단행본 만화 에피소드 클론 영화 시리즈 애니메이션,스타워즈 STAR WARS 클론워즈 어드벤처 5,2008,만화,그래픽노블,판타지,판타지,애니북스,DARK HORSE BOOKS,애니북스


# allbookdata저장
## feature에이용될 값중 빈값 모두 제거했음

In [ ]:
vector_df.to_csv("xdeep_allbook.csv",index=False)

In [34]:
##########################################################################################

In [7]:
xdeep_allbook = pd.read_csv('xdeep_allbook.csv')

In [8]:
xdeep_allbook

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,kwd,title.1,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher
0,-0.196637,0.020864,0.350972,0.192017,-0.206795,-0.054141,0.136589,-0.118211,-0.104625,-0.071694,...,성경 교양서 가톨릭 금장 이해 색인,성경(46판 2단 금장색인) (BB6 BH) (250130),2006.0,종교,가톨릭,가톨릭성서,가톨릭성서,가톨릭출판사 편집부가톨릭출판사,가톨릭출판사 편집부,가톨릭출판사
1,-0.196837,-0.122482,0.348215,0.117282,-0.099319,-0.070873,0.216196,0.005626,-0.180764,-0.030805,...,교양서 불교 남북 연구 이해 율장 비교,남북전육부 율장비교연구(3판),1976.0,종교,불교,불교철학사상,불교철학사상,이지관가산문고,이지관,가산문고
2,-0.133873,-0.069763,0.103495,0.018553,-0.203616,-0.056440,0.215832,0.082005,-0.384968,-0.267825,...,감각,제7감각을 기르자,1994.0,인문,교육학,교수학습법,학습법,죤 니스벳 자넷 셕스미쓰원미사,"죤 니스벳, 자넷 셕스미쓰",원미사
3,-0.079808,0.125609,-0.159979,-0.060015,0.081533,-0.066703,0.242454,0.133134,-0.482992,0.177329,...,노동 디오니소스 교양 전공 전문 이해 대상,디오니소스의 노동 1,1996.0,정치사회,사회문제복지,노동문제,노동문제일반,안또니오 네그리갈무리,안또니오 네그리,갈무리
4,0.172614,-0.076462,0.229195,0.080075,0.038470,0.082090,0.004960,0.132072,-0.410798,-0.079386,...,아침 햇살,아침햇살이 그립다,2001.0,시에세이,한국시,현대시,현대시,표성배갈무리,표성배,갈무리
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975889,-0.057828,0.067116,-0.023779,0.157665,-0.401717,-0.264535,0.191748,-0.234187,-0.223327,0.101256,...,신일숙 아르 미안 전집 환상,아르미안의 네 딸들 10(신일숙 환상전집 1),2008.0,만화,로맨스만화,로맨스만화,로맨스만화,신일숙학산문화사,신일숙,학산문화사
975890,-0.163352,-0.090657,0.200129,0.268317,-0.316147,0.228996,0.366973,-0.162490,-0.195312,0.101570,...,완결,너에게 간다 9(완결),2009.0,만화,로맨스만화,로맨스만화,로맨스만화,한유랑삼양출판사,한유랑,삼양출판사
975891,-0.171152,0.075161,0.338542,0.336318,-0.265556,-0.217217,0.132284,0.056170,-0.285489,0.117279,...,와인 포켓북 문고판 여행,즐거운 와인 여행 2(포켓북(문고판)),2008.0,요리,와인커피음료,와인,와인,이동현김정,이동현,김&정
975892,-0.167075,0.104481,-0.072516,0.121800,-0.037745,-0.094742,0.241081,0.018632,-0.512108,0.105987,...,작품 소설 스토리 단행본 만화 에피소드 클론 영화 시리즈 애니메이션,스타워즈 STAR WARS 클론워즈 어드벤처 5,2008.0,만화,그래픽노블,판타지,판타지,애니북스,DARK HORSE BOOKS,애니북스


# User log 데이터전처리

In [9]:
search

,customer_type,customer_idx,param,created_at
0,user,5147,talk to me in korean,2022-06-28 22:35:11
1,user,5151,허리,2022-06-29 04:02:43
2,user,5151,허리 운동,2022-06-29 04:02:53
3,user,5151,허리 운동,2022-06-29 04:09:23
4,user,5151,허리,2022-06-29 04:09:57
...,...,...,...,...
1148,company_emp,99,근대의 경계에서,2022-09-20 03:03:44
1149,user,5978,집에 있는데도 집에 가고 싶어,2022-09-20 17:10:17
1150,user,5978,82년,2022-09-20 17:10:42
1151,user,5978,82년생 김지영,2022-09-20 17:11:18


In [10]:
user_log = pd.concat([detail_title,wish_title])

In [11]:
# 검색기록을 모두 합치기위한 전처리

In [12]:
user_log.drop(columns = ['Unnamed: 0','date','isbn'],inplace=True)

In [13]:
user_log.rename(columns={'title':'param'},inplace=True)

In [14]:
user_log

,customer_type,customer_idx,created_at,param
0,user,53,2022-06-28 10:58:06,Anytime Korean Beginning 1
1,user,5128,2022-06-28 12:20:45,Anytime Korean Beginning 1
2,user,5080,2022-06-28 12:49:22,Anytime Korean Beginning 1
3,user,5109,2022-06-29 09:30:47,Anytime Korean Beginning 1
4,user,5109,2022-06-29 09:52:44,Anytime Korean Beginning 1
...,...,...,...,...
48,user,5768,2022-08-29 18:36:41,김 비서가 왜 그럴까 2
49,user,5768,2022-08-29 18:36:44,김 비서가 왜 그럴까 1
50,user,5768,2022-08-29 18:47:17,김 비서가 왜 그럴까 1
51,user,5768,2022-08-29 18:36:46,김 비서가 왜 그럴까 2


In [15]:
user_log_all = pd.concat([search,user_log])

In [16]:
user_log_all

,customer_type,customer_idx,param,created_at
0,user,5147,talk to me in korean,2022-06-28 22:35:11
1,user,5151,허리,2022-06-29 04:02:43
2,user,5151,허리 운동,2022-06-29 04:02:53
3,user,5151,허리 운동,2022-06-29 04:09:23
4,user,5151,허리,2022-06-29 04:09:57
...,...,...,...,...
48,user,5768,김 비서가 왜 그럴까 2,2022-08-29 18:36:41
49,user,5768,김 비서가 왜 그럴까 1,2022-08-29 18:36:44
50,user,5768,김 비서가 왜 그럴까 1,2022-08-29 18:47:17
51,user,5768,김 비서가 왜 그럴까 2,2022-08-29 18:36:46


In [17]:
user_log_all['customer_idx'].nunique()

519

In [18]:
customer_idx = 5151
is_user = user_log_all['customer_idx'] == customer_idx

In [19]:
is_user

0     False
1      True
2      True
3      True
4      True
      ...  
48    False
49    False
50    False
51    False
52    False
Name: customer_idx, Length: 2922, dtype: bool

In [20]:
user_search = user_log_all[is_user]

In [21]:
user_search

,customer_type,customer_idx,param,created_at
1,user,5151,허리,2022-06-29 04:02:43
2,user,5151,허리 운동,2022-06-29 04:02:53
3,user,5151,허리 운동,2022-06-29 04:09:23
4,user,5151,허리,2022-06-29 04:09:57
5,user,5151,허리디스크,2022-06-29 04:18:51
561,user,5151,요통 정복: 요통 환자도 해야 하는 허리 운동(고도일의 평생 척추 관절 건강 프로젝...,2022-06-29 04:02:58
562,user,5151,요통 정복: 요통 환자도 해야 하는 허리 운동(고도일의 평생 척추 관절 건강 프로젝...,2022-06-29 04:09:52
563,user,5151,모두를 위한 허리 교과서,2022-06-29 04:03:16
564,user,5151,모두를 위한 허리 교과서,2022-06-29 04:14:31
565,user,5151,모두를 위한 허리 교과서,2022-06-29 04:16:29


In [22]:
param_list = user_search['param'].values.tolist()
param_list # keyword검색값, isbn검색값

['허리',
 '허리 운동',
 '허리 운동',
 '허리',
 '허리디스크',
 '요통 정복: 요통 환자도 해야 하는 허리 운동(고도일의 평생 척추 관절 건강 프로젝트 1)(Paperback)',
 '요통 정복: 요통 환자도 해야 하는 허리 운동(고도일의 평생 척추 관절 건강 프로젝트 1)(Paperback)',
 '모두를 위한 허리 교과서',
 '모두를 위한 허리 교과서',
 '모두를 위한 허리 교과서',
 '모두를 위한 허리 교과서',
 '허리 좀 펴고 삽시다',
 '허리, 무릎 관절 통증 수술 없이 고칠수 있다',
 '허리 목 통증재발 단계별 맞춤운동',
 '모두를 위한 허리 교과서(큰글자도서)(리더스원)',
 '모두를 위한 허리 교과서(큰글자도서)(리더스원)',
 '허리디스크 수술 없이 낫기']

In [23]:
word = ("0", "1","2","3","4","5","6","7","8","9")
param_only = []
param_isbn = []

In [24]:
def param_filtering(x):
    if x.startswith(word) and x.endswith(word):
        param_isbn.append(x)
    else:
        param_only.append(x)

In [25]:
# 사용자 검색어(param)를 키워드 or isbn으로 분리
user_search['param'].apply(param_filtering)

1      None
2      None
3      None
4      None
5      None
561    None
562    None
563    None
564    None
565    None
566    None
567    None
568    None
569    None
570    None
571    None
572    None
Name: param, dtype: object

In [26]:
from functools import reduce
import operator

In [27]:
 # param_isbn검색 있을시, param_isbn을 띄어쓰기 기준으로 리스트로 만듬.
isbn_after_filter_1 = []
isbn_int = []

if param_isbn != []:
    for x in param_isbn:
        x_split = x.split(' ')
        isbn_after_filter_1.append(x_split)
    
    isbn_after_filter_2 = list(reduce(operator.add, isbn_after_filter_1))
    
    isbn_after_filter_2 = [word.strip('X') for word in isbn_after_filter_2 ] # X 제거
    isbn_after_filter_2 = [word.strip('-') for word in isbn_after_filter_2 ] # - 제거
    isbn_after_filter_2 = [word.strip(',') for word in isbn_after_filter_2 ] # , 제거
    isbn_after_filter_2 = [word.strip('.') for word in isbn_after_filter_2 ] # . 제거
    isbn_after_filter_2 = list(filter(None, isbn_after_filter_2)) # ''제거
    
    # - 제거
    isbn_after_filter_3 = []
    for a in isbn_after_filter_2:
        isbn_after_filter_3.append(a.replace('-',''))
        
    isbn_int = list(map(int,isbn_after_filter_3)) # 정수로 변환

In [28]:
param_only[0]

'허리'

# 사용자 검색 책 목록 찾기

In [29]:
xdeep_allbook.dropna(inplace=True)

In [34]:
searchdata=xdeep_allbook[:1]
searchdata

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,kwd,title.1,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher
0,-0.196637,0.020864,0.350972,0.192017,-0.206795,-0.054141,0.136589,-0.118211,-0.104625,-0.071694,...,성경 교양서 가톨릭 금장 이해 색인,성경(46판 2단 금장색인) (BB6 BH) (250130),2006.0,종교,가톨릭,가톨릭성서,가톨릭성서,가톨릭출판사 편집부가톨릭출판사,가톨릭출판사 편집부,가톨릭출판사


In [35]:

for i,param in enumerate(param_only):
    param_mask = xdeep_allbook['title'].str.contains(param)
    search_param = xdeep_allbook[param_mask] # search_param ; 사용자가 검색한 kwd 책 리스트
    isbn_mask = xdeep_allbook['isbn'].isin(isbn_int)
    search_isbn = xdeep_allbook[isbn_mask] # search_isbn ; 사용자가 검색한 isbn 책 리스트

    if i==1: # i==1일때는 첫번째 데이터프레임을 갖고옴
        tmp = pd.concat([search_param,search_isbn])
        searchdata = tmp
    else: # i==2부터는 첫번째 데이터프레임에 concat
        tmp = pd.concat([search_param,search_isbn])
        searchdata = pd.concat([searchdata,tmp])
    

################################

/tmp/ipykernel_4791/676707268.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  param_mask = xdeep_allbook['title'].str.contains(param)


In [36]:
searchdata # 사용자가 검색한 책들

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,kwd,title.1,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher
388059,0.063206,-0.042553,-0.000355,-0.005227,0.060102,-0.138579,0.005916,0.006686,0.022245,0.024140,...,신경외과 척추 관절 통증 근육 디스크 골반 수술 심부 허벅지,요통 정복: 요통 환자도 해야 하는 허리 운동(고도일의 평생 척추 관절 건강 프로젝...,2016.0,건강,질병치료예방,요통,요통,고도일푸른솔,고도일,푸른솔
555642,-0.129968,-0.226031,0.180187,0.404691,0.032131,-0.203480,-0.016048,0.174368,-0.391859,0.098091,...,척추 심층부 디스크 운동 정신 트레이너 허리 프로 통증 동작,기적의 허리 운동법,2020.0,건강,운동트레이닝,스트레칭,스트레칭,질베르 보보프로제,질베르 보보,프로제
388059,0.063206,-0.042553,-0.000355,-0.005227,0.060102,-0.138579,0.005916,0.006686,0.022245,0.024140,...,신경외과 척추 관절 통증 근육 디스크 골반 수술 심부 허벅지,요통 정복: 요통 환자도 해야 하는 허리 운동(고도일의 평생 척추 관절 건강 프로젝...,2016.0,건강,질병치료예방,요통,요통,고도일푸른솔,고도일,푸른솔
555642,-0.129968,-0.226031,0.180187,0.404691,0.032131,-0.203480,-0.016048,0.174368,-0.391859,0.098091,...,척추 심층부 디스크 운동 정신 트레이너 허리 프로 통증 동작,기적의 허리 운동법,2020.0,건강,운동트레이닝,스트레칭,스트레칭,질베르 보보프로제,질베르 보보,프로제
4139,0.049353,0.266408,-0.020518,0.372852,-0.170882,-0.220941,0.222820,0.135584,0.106027,0.012247,...,허리 어깨 통증 세트,"통증홈트(목, 어깨, 허리) 세트",2017.0,건강,건강일반,건강상식,건강상식,남세희중앙북스,남세희,중앙북스
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916627,-0.077322,0.015766,0.265089,0.190867,-0.191979,-0.004557,0.269276,0.136133,-0.240786,0.101439,...,척추 추간판 고관절 디스크 통증 생리 학적 신체 허리 공부,모두를 위한 허리 교과서(큰글자도서)(리더스원),2022.0,건강,질병치료예방,요통,요통,안병택블루무스,안병택,블루무스
860622,-0.100891,0.051394,-0.171432,-0.049345,0.135024,-0.107283,0.264321,-0.045300,-0.480863,0.135858,...,척추관 척추 골반 체조 통증 고관절 허리뼈 근육 가슴 고질병,허리 좀 펴고 삽시다,2021.0,건강,질병치료예방,디스크척추,디스크척추,기쿠치 신이치 가네오카 고지 와타라이 고지 요시하라 기요시 와타나베 고타포레스트북스,"기쿠치 신이치, 가네오카 고지, 와타라이 고지, 요시하라 기요시, 와타나베 고타",포레스트북스
608828,-0.086349,-0.082776,0.262079,0.261764,-0.134162,-0.111146,0.291015,0.079112,-0.227754,0.023585,...,관절 무릎 통증 허리 수술,"허리, 무릎 관절 통증 수술 없이 고칠수 있다",2018.0,건강,질병치료예방,디스크척추,디스크척추,이건목도토리하우스,이건목,도토리하우스
672066,-0.161285,-0.032756,0.471075,0.176741,-0.311365,-0.125080,0.291044,0.027319,-0.304579,-0.000942,...,허리 운동 통증 어깨 자세 재활 강화 단계 요통 기구,허리 목 통증재발 단계별 맞춤운동,2021.0,건강,운동트레이닝,스트레칭,스트레칭,김건도 이광수청풍출판사,"김건도, 이광수",청풍출판사


In [37]:
searchdata.drop(['title.1'], axis=1,inplace=True)

In [38]:
searchdata['Cn_3'].dropna(inplace=True) # Cn_3 결측값 제거
searchdata.drop_duplicates(['isbn'],inplace=True) # 중복제거

In [39]:
user_log_cn3 = searchdata['Cn_3'].values.tolist() # user_log_cn3 ; user가 검색한 책들의 Cn3_list

In [40]:
user_log_cn3

['요통',
 '스트레칭',
 '건강상식',
 '창작동화',
 '기도설교전도',
 '전래동화',
 '과학동식물환경',
 '로맨스소설',
 '한국소설일반',
 '한국소설일반',
 '건강상식',
 '한국소설일반',
 '디스크척추',
 '과학동식물환경',
 '요통',
 '요통',
 '디스크척추',
 '로맨스소설',
 '디스크척추',
 '다이어트일반',
 '현대시',
 '기독교성가집',
 '현대시',
 '판타지소설',
 '판타지소설',
 '판타지소설',
 '판타지소설',
 '판타지소설',
 '디스크척추',
 '요통',
 '디스크척추',
 '디스크척추',
 '골프',
 '기초의학',
 '디스크척추',
 '다이어트일반',
 '디스크척추',
 '디스크척추',
 '디스크척추',
 '디스크척추',
 '외과',
 '외과',
 '외과',
 '디스크척추',
 '한국에세이',
 '디스크척추',
 '디스크척추',
 '디스크척추',
 '디스크척추',
 '그림책',
 '디스크척추',
 '디스크척추',
 '한국에세이',
 '디스크척추',
 '요통',
 '디스크척추',
 '동화책',
 '외과',
 '디스크척추',
 '요통',
 '건강이야기',
 '신앙생활',
 '스트레칭',
 '외과',
 '만화애니메이션',
 '동화책',
 '요통',
 '기공체조',
 '디스크척추',
 '일본에세이',
 '신앙생활',
 '테니스배드민턴',
 '디스크척추',
 '한국그림책',
 '과학동식물환경',
 '디스크척추',
 '현대시',
 '건강이야기',
 '시에세이문고 일반',
 '스포츠의학',
 '외과',
 '다이어트일반',
 '외과',
 '디스크척추',
 '헬스웨이트트레이닝',
 '다이어트일반',
 '디스크척추',
 '디스크척추',
 '백과예능',
 '동화책',
 '스트레칭',
 '디스크척추',
 '그림책',
 '디스크척추',
 '디스크척추',
 '디스크척추',
 '한국에세이',
 '현대시',
 '동화책',
 '자기관리처세',
 '대체의학',
 '어린이문학',
 '디스크척추',
 '기초의학',
 '디스크척추',
 

In [41]:
from collections import Counter
counter = Counter(user_log_cn3)

In [42]:
keycount = dict(counter)

In [43]:
keycount

{'요통': 9,
 '스트레칭': 3,
 '건강상식': 2,
 '창작동화': 1,
 '기도설교전도': 1,
 '전래동화': 1,
 '과학동식물환경': 3,
 '로맨스소설': 2,
 '한국소설일반': 3,
 '디스크척추': 36,
 '다이어트일반': 4,
 '현대시': 5,
 '기독교성가집': 1,
 '판타지소설': 5,
 '골프': 1,
 '기초의학': 3,
 '외과': 7,
 '한국에세이': 4,
 '그림책': 2,
 '동화책': 4,
 '건강이야기': 4,
 '신앙생활': 2,
 '만화애니메이션': 1,
 '기공체조': 1,
 '일본에세이': 1,
 '테니스배드민턴': 1,
 '한국그림책': 1,
 '시에세이문고 일반': 1,
 '스포츠의학': 1,
 '헬스웨이트트레이닝': 1,
 '백과예능': 1,
 '자기관리처세': 1,
 '대체의학': 1,
 '어린이문학': 1,
 '역사이야기': 1,
 '외국그림책': 1,
 '학습만화': 1}

In [44]:
sorted_cn3 = sorted(keycount.items(), reverse=True,key=lambda item: item[1])

In [45]:
cn3_user_interestd = dict(sorted_cn3[:3])

In [46]:
cn3_user_interestd

{'디스크척추': 36, '요통': 9, '외과': 7}

In [47]:
like_cn3 = []

In [48]:
for key,val in cn3_user_interestd.items():
    if(val>1):
        like_cn3.append(key)

In [49]:
like_cn3 # 유저가 가장 관심있는 카테고리

['디스크척추', '요통', '외과']

In [ ]:
# idx=0
# for key,val in keycount.items():
#     if(val>4):
#         user_log_cn3_filter.append(key)

(방법고민중,,)
# 사용자가 검색관련 Cn_3중 높은 빈도를 차지하는 세가지 Cn_3을 기준으로,
# title이 해당 Cn_3을 포함하는 것을 재검색 후 Cn_3을 user_cn3에 추가?
# or user_cn3에 title이 해당Cn_3을 포함하는 것을 추가하여 모두 저장?

# or 해당 like_Cn3을 기준으로 라벨링 진행...?

In [52]:
# or 사용자가 관심있는 카테고리는 해당카테고리의 cn_2에 해당하는 책들까지 포함해 준다.

In [58]:
userlog_cn3_mask = xdeep_allbook['Cn_3'].isin(like_cn3)
search_param = xdeep_allbook[userlog_cn3_mask]
user_cn2 = search_param['Cn_2'].values.tolist()
userlog_cn2_mask = xdeep_allbook['Cn_2'].isin(user_cn2)
user_cn2_param = xdeep_allbook[userlog_cn2_mask]

In [59]:
user_cn2_param

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,kwd,title.1,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher
49,0.004311,0.011868,0.291177,0.101379,-0.026793,-0.098076,0.338463,-0.043485,-0.177415,0.114274,...,교재 전공 증기,복증기람(상.하),1981.0,기술공학,의학,한의학,한의학의경,계축문화사,稻葉文禮,계축문화사
51,-0.000611,-0.010616,0.214332,0.129003,0.005000,-0.225164,0.291314,-0.013412,-0.148166,0.063869,...,교재 단학 문진 전공,불문진단학(전)(알기쉬운),1976.0,기술공학,의학,한의학,한의학이론,변성훈계축문화사,변성훈,계축문화사
52,-0.044766,0.020373,0.309745,0.127479,0.034779,-0.125481,0.277891,-0.049633,-0.168874,0.107704,...,교재 전공 후세 소허 방학,소허후세처방학,1993.0,기술공학,의학,한의학,각과한의학,조세형계축문화사,조세형,계축문화사
55,-0.074932,0.052626,0.358192,0.243049,-0.150164,-0.125322,0.292745,0.048681,-0.126492,0.065493,...,본초학 교재 전공 한국,한국본초학,1981.0,기술공학,의학,한의학,본초학,육창수 외계축문화사,육창수 외,계축문화사
61,-0.014488,0.083864,0.301727,0.169021,-0.003764,-0.183756,0.349898,-0.087025,-0.128612,0.050771,...,배양 교재 세포 조직 전공,조직세포배양,1997.0,기술공학,의학,기초의학,조직학세포학,박승택계축문화사,박승택,계축문화사
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967888,0.066758,-0.095670,0.206293,0.133527,0.099845,-0.012704,-0.009222,0.196242,-0.395629,-0.059810,...,개정판 서술 실습 출제 임상 영상 연습 학실 학교 챕터,의료영상학실습 2(제3판)(개정판 2판),2010.0,기술공학,의학,기초의학,초음파방사선,의료영상기술연구회청구문화사,의료영상기술연구회,청구문화사
973814,0.063696,-0.001096,0.263253,0.043750,0.119434,0.135114,-0.031265,0.285856,-0.296005,-0.063525,...,자연 기법 치부 부합,NAT 자연에 부합하는 WAX-UP 기법 Vol 전치부,2004.0,기술공학,의학,치과,구강학구강진단학,대한나래출판사,DIETER SCHULZ,대한나래출판사
974695,-0.061199,-0.168128,0.471005,-0.114436,-0.051186,0.029746,0.073259,-0.169076,-0.056949,0.153324,...,양장본 구대성,침구대성 1(양장본 HardCover),2007.0,기술공학,의학,한의학,침구경락경혈,소재진대성의학사,소재진,대성의학사
974744,-0.314074,-0.113504,0.121787,0.045765,-0.533063,-0.313713,0.267695,0.332422,-0.005922,-0.096232,...,악교정수술 탈회 상악 피질 교정지 임상 절단술 지견 치료 교정,연세임상교정(2016 Vol 23),2016.0,기술공학,의학,치과,교정학,연세치대교정과학교실두개안면기형연구소대한나래출판사,연세치대교정과학교실두개안면기형연구소,대한나래출판사


In [104]:
########################################################################

# user log 를 바탕으로 negative / positive data 생성
# -user가 검색한 책 카테고리에 속한 것은 1로 labeling
# -user가 검색하지 않은 책 카테고리에 속한 것은 0으로 labeling

In [65]:
userlog_cn3_mask = xdeep_allbook['Cn_3'].isin(user_log_cn3)
negative_data = xdeep_allbook[~userlog_cn3_mask]
positive_data = xdeep_allbook[userlog_cn3_mask]

In [66]:
print(len(positive_data))
print(len(negative_data))

221902
753991


In [67]:
positive_data

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,kwd,title.1,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher
4,0.172614,-0.076462,0.229195,0.080075,0.038470,0.082090,0.004960,0.132072,-0.410798,-0.079386,...,아침 햇살,아침햇살이 그립다,2001.0,시에세이,한국시,현대시,현대시,표성배갈무리,표성배,갈무리
47,-0.138805,-0.006184,0.330374,0.136741,-0.140643,-0.079239,0.139073,0.097032,-0.194882,-0.104079,...,교양서 기독교 행복 가정 이해,행복한 가정,1990.0,종교,기독교개신교,신앙생활,데이트결혼가정,곽선희계몽사,곽선희,계몽사
48,-0.313524,-0.169985,0.310959,0.356585,-0.210815,-0.061292,0.258574,0.004373,-0.147370,0.038922,...,교양서 교집 참회 기독교 은총 이해,참회의 은총(설교집 11),1990.0,종교,기독교개신교,기도설교전도,설교학,곽선희계몽사,곽선희,계몽사
50,-0.274005,-0.157250,0.534772,0.416390,-0.312562,-0.060082,0.064082,0.091576,0.064364,-0.266678,...,교양서 복음 고린도 전서 기독교 이해 능력,복음의 능력(고린도전서강해),1999.0,종교,기독교개신교,기도설교전도,설교학,곽선희계몽문화사,곽선희,계몽문화사
53,-0.203387,0.109809,0.475916,0.217084,-0.048670,0.023162,0.100353,-0.015469,-0.146895,-0.063362,...,교집 관심 궁극,궁극적 관심(설교집 12),1991.0,종교,기독교개신교,기도설교전도,설교학,곽선희계몽사,곽선희,계몽사
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975844,-0.239055,-0.082591,0.096957,-0.063444,-0.401151,-0.401449,0.378712,0.199635,0.060089,0.113318,...,서정주 민음사 문학상 국문학사 시인 유작 문학 전집 시집 박목월,그대에게 가는 길 2(임명조 시전집),2008.0,시에세이,한국시,현대시,현대시,임영조천년의시작,임영조,천년의시작
975855,-0.200095,0.030464,0.227539,0.210373,-0.350274,-0.105664,0.387696,0.185047,-0.452608,-0.030626,...,문학사 시인 문학 평론가 소설가 작가 소설 명작 장명수 작품,김훈 박래부의 문학기행 하나(개정판),2007.0,시에세이,나라별 에세이,한국에세이,한국에세이,김훈 박래부따뜻한손,"김훈, 박래부",따뜻한손
975871,-0.188575,0.050568,-0.154235,-0.146561,0.155929,-0.067542,0.309226,0.055504,-0.509148,0.252721,...,작품집 문학 널문 회원 아침,내 앞에 열린 아침 7(문학저널문인회원 작품집 : 시 5),2007.0,시에세이,한국시,현대시,현대시,권기만 권오정 권정숙 김미선 김세라엠아이지,"권기만, 권오정, 권정숙, 김미선, 김세라",엠아이지
975872,-0.231884,-0.014245,0.109123,-0.032490,-0.486659,-0.303347,0.216877,0.246601,0.034002,0.079589,...,양장본 세계 마을 만일,세계가 만일 100명의 마을이라면 2(양장본 HardCover),2003.0,시에세이,나라별 에세이,일본에세이,일본에세이,이케다 가요코국일미디어,이케다 가요코,국일미디어


In [68]:
# data labeling
positive_data['label']=1
negative_data['label']=0

/tmp/ipykernel_4791/534838286.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_data['label']=1
/tmp/ipykernel_4791/534838286.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_data['label']=0


# 최종 data

In [69]:
data = pd.concat([positive_data,negative_data])
data # final data

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,title.1,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher,label
4,0.172614,-0.076462,0.229195,0.080075,0.038470,0.082090,0.004960,0.132072,-0.410798,-0.079386,...,아침햇살이 그립다,2001.0,시에세이,한국시,현대시,현대시,표성배갈무리,표성배,갈무리,1
47,-0.138805,-0.006184,0.330374,0.136741,-0.140643,-0.079239,0.139073,0.097032,-0.194882,-0.104079,...,행복한 가정,1990.0,종교,기독교개신교,신앙생활,데이트결혼가정,곽선희계몽사,곽선희,계몽사,1
48,-0.313524,-0.169985,0.310959,0.356585,-0.210815,-0.061292,0.258574,0.004373,-0.147370,0.038922,...,참회의 은총(설교집 11),1990.0,종교,기독교개신교,기도설교전도,설교학,곽선희계몽사,곽선희,계몽사,1
50,-0.274005,-0.157250,0.534772,0.416390,-0.312562,-0.060082,0.064082,0.091576,0.064364,-0.266678,...,복음의 능력(고린도전서강해),1999.0,종교,기독교개신교,기도설교전도,설교학,곽선희계몽문화사,곽선희,계몽문화사,1
53,-0.203387,0.109809,0.475916,0.217084,-0.048670,0.023162,0.100353,-0.015469,-0.146895,-0.063362,...,궁극적 관심(설교집 12),1991.0,종교,기독교개신교,기도설교전도,설교학,곽선희계몽사,곽선희,계몽사,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975889,-0.057828,0.067116,-0.023779,0.157665,-0.401717,-0.264535,0.191748,-0.234187,-0.223327,0.101256,...,아르미안의 네 딸들 10(신일숙 환상전집 1),2008.0,만화,로맨스만화,로맨스만화,로맨스만화,신일숙학산문화사,신일숙,학산문화사,0
975890,-0.163352,-0.090657,0.200129,0.268317,-0.316147,0.228996,0.366973,-0.162490,-0.195312,0.101570,...,너에게 간다 9(완결),2009.0,만화,로맨스만화,로맨스만화,로맨스만화,한유랑삼양출판사,한유랑,삼양출판사,0
975891,-0.171152,0.075161,0.338542,0.336318,-0.265556,-0.217217,0.132284,0.056170,-0.285489,0.117279,...,즐거운 와인 여행 2(포켓북(문고판)),2008.0,요리,와인커피음료,와인,와인,이동현김정,이동현,김&정,0
975892,-0.167075,0.104481,-0.072516,0.121800,-0.037745,-0.094742,0.241081,0.018632,-0.512108,0.105987,...,스타워즈 STAR WARS 클론워즈 어드벤처 5,2008.0,만화,그래픽노블,판타지,판타지,애니북스,DARK HORSE BOOKS,애니북스,0


In [70]:
data.drop_duplicates(['isbn'],inplace=True) # 중복제거

In [71]:
data

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,title.1,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher,label
4,0.172614,-0.076462,0.229195,0.080075,0.038470,0.082090,0.004960,0.132072,-0.410798,-0.079386,...,아침햇살이 그립다,2001.0,시에세이,한국시,현대시,현대시,표성배갈무리,표성배,갈무리,1
47,-0.138805,-0.006184,0.330374,0.136741,-0.140643,-0.079239,0.139073,0.097032,-0.194882,-0.104079,...,행복한 가정,1990.0,종교,기독교개신교,신앙생활,데이트결혼가정,곽선희계몽사,곽선희,계몽사,1
48,-0.313524,-0.169985,0.310959,0.356585,-0.210815,-0.061292,0.258574,0.004373,-0.147370,0.038922,...,참회의 은총(설교집 11),1990.0,종교,기독교개신교,기도설교전도,설교학,곽선희계몽사,곽선희,계몽사,1
50,-0.274005,-0.157250,0.534772,0.416390,-0.312562,-0.060082,0.064082,0.091576,0.064364,-0.266678,...,복음의 능력(고린도전서강해),1999.0,종교,기독교개신교,기도설교전도,설교학,곽선희계몽문화사,곽선희,계몽문화사,1
53,-0.203387,0.109809,0.475916,0.217084,-0.048670,0.023162,0.100353,-0.015469,-0.146895,-0.063362,...,궁극적 관심(설교집 12),1991.0,종교,기독교개신교,기도설교전도,설교학,곽선희계몽사,곽선희,계몽사,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975889,-0.057828,0.067116,-0.023779,0.157665,-0.401717,-0.264535,0.191748,-0.234187,-0.223327,0.101256,...,아르미안의 네 딸들 10(신일숙 환상전집 1),2008.0,만화,로맨스만화,로맨스만화,로맨스만화,신일숙학산문화사,신일숙,학산문화사,0
975890,-0.163352,-0.090657,0.200129,0.268317,-0.316147,0.228996,0.366973,-0.162490,-0.195312,0.101570,...,너에게 간다 9(완결),2009.0,만화,로맨스만화,로맨스만화,로맨스만화,한유랑삼양출판사,한유랑,삼양출판사,0
975891,-0.171152,0.075161,0.338542,0.336318,-0.265556,-0.217217,0.132284,0.056170,-0.285489,0.117279,...,즐거운 와인 여행 2(포켓북(문고판)),2008.0,요리,와인커피음료,와인,와인,이동현김정,이동현,김&정,0
975892,-0.167075,0.104481,-0.072516,0.121800,-0.037745,-0.094742,0.241081,0.018632,-0.512108,0.105987,...,스타워즈 STAR WARS 클론워즈 어드벤처 5,2008.0,만화,그래픽노블,판타지,판타지,애니북스,DARK HORSE BOOKS,애니북스,0


In [72]:
data.columns

Index(['catev_0', 'catev_1', 'catev_2', 'catev_3', 'catev_4', 'catev_5',
       'catev_6', 'catev_7', 'catev_8', 'catev_9',
       ...
       'title.1', 'date', 'Cn_1', 'Cn_2', 'Cn_3', 'Cn_4', 'author_publisher',
       'author', 'publisher', 'label'],
      dtype='object', length=349)

In [73]:
# categorising the features into sparse/dense feaure set
sparse_features = ["isbn","author", "publisher"]
dense_features = ['catev_' + str(i) for i in range(1,80)]

In [74]:
dense_features2 = [str(i) for i in range(1,80)]

In [75]:
dense_features.extend(dense_features2)

In [76]:
dense_features

['catev_1',
 'catev_2',
 'catev_3',
 'catev_4',
 'catev_5',
 'catev_6',
 'catev_7',
 'catev_8',
 'catev_9',
 'catev_10',
 'catev_11',
 'catev_12',
 'catev_13',
 'catev_14',
 'catev_15',
 'catev_16',
 'catev_17',
 'catev_18',
 'catev_19',
 'catev_20',
 'catev_21',
 'catev_22',
 'catev_23',
 'catev_24',
 'catev_25',
 'catev_26',
 'catev_27',
 'catev_28',
 'catev_29',
 'catev_30',
 'catev_31',
 'catev_32',
 'catev_33',
 'catev_34',
 'catev_35',
 'catev_36',
 'catev_37',
 'catev_38',
 'catev_39',
 'catev_40',
 'catev_41',
 'catev_42',
 'catev_43',
 'catev_44',
 'catev_45',
 'catev_46',
 'catev_47',
 'catev_48',
 'catev_49',
 'catev_50',
 'catev_51',
 'catev_52',
 'catev_53',
 'catev_54',
 'catev_55',
 'catev_56',
 'catev_57',
 'catev_58',
 'catev_59',
 'catev_60',
 'catev_61',
 'catev_62',
 'catev_63',
 'catev_64',
 'catev_65',
 'catev_66',
 'catev_67',
 'catev_68',
 'catev_69',
 'catev_70',
 'catev_71',
 'catev_72',
 'catev_73',
 'catev_74',
 'catev_75',
 'catev_76',
 'catev_77',
 'catev_

In [77]:
# data inputation for missing values
data[sparse_features] = data[sparse_features].fillna('-1',)

In [78]:
# creating target variable
target = ['label']

In [79]:
# encoding function  // labelencoder ; 문자를 0부터 증가하는 정수형 숫자로 바꿔주는기능을 제공
def encoding(data,feat,encoder):
    data[feat] = encoder.fit_transform(data[feat])

In [80]:
# encoding for categorical feautures
[encoding(data, feat, LabelEncoder()) for feat in sparse_features]

[None, None, None]

In [81]:
# importing libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from deepctr.models import xDeepFM
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names

In [82]:
# creating a 4 bit embedding for every sparse feature
sparse_feature_columns = [SparseFeat(feat,vocabulary_size = data[feat].nunique(), embedding_dim=4)
                         for i,feat in enumerate(sparse_features)]

In [83]:
# Creating a dense feat
dense_feature_columns = [DenseFeat(feat,1) for feat in dense_features]

In [84]:
# features to be used for dnn part of xdeepfm
dnn_feature_columns = sparse_feature_columns + dense_feature_columns
# features to be used for linear part of xdeepfm
linear_feature_columns = sparse_feature_columns + dense_feature_columns

In [85]:
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
feature_names

['isbn',
 'author',
 'publisher',
 'catev_1',
 'catev_2',
 'catev_3',
 'catev_4',
 'catev_5',
 'catev_6',
 'catev_7',
 'catev_8',
 'catev_9',
 'catev_10',
 'catev_11',
 'catev_12',
 'catev_13',
 'catev_14',
 'catev_15',
 'catev_16',
 'catev_17',
 'catev_18',
 'catev_19',
 'catev_20',
 'catev_21',
 'catev_22',
 'catev_23',
 'catev_24',
 'catev_25',
 'catev_26',
 'catev_27',
 'catev_28',
 'catev_29',
 'catev_30',
 'catev_31',
 'catev_32',
 'catev_33',
 'catev_34',
 'catev_35',
 'catev_36',
 'catev_37',
 'catev_38',
 'catev_39',
 'catev_40',
 'catev_41',
 'catev_42',
 'catev_43',
 'catev_44',
 'catev_45',
 'catev_46',
 'catev_47',
 'catev_48',
 'catev_49',
 'catev_50',
 'catev_51',
 'catev_52',
 'catev_53',
 'catev_54',
 'catev_55',
 'catev_56',
 'catev_57',
 'catev_58',
 'catev_59',
 'catev_60',
 'catev_61',
 'catev_62',
 'catev_63',
 'catev_64',
 'catev_65',
 'catev_66',
 'catev_67',
 'catev_68',
 'catev_69',
 'catev_70',
 'catev_71',
 'catev_72',
 'catev_73',
 'catev_74',
 'catev_75',


In [86]:
# creating train test splits
train, test = train_test_split(data, test_size = 0.2) # 200*0.2 = 40개를 test_dataset으로 분리
train_model_input = {name: train[name].values for name in feature_names}
test_model_input = {name: test[name].values for name in feature_names}

In [87]:
from deepctr.models.xdeepfm import xDeepFM

In [88]:
model = xDeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256),
cin_layer_size=(128, 128),
cin_split_half=True, cin_activation='relu'
,l2_reg_linear=1e-05,
l2_reg_embedding=1e-05, l2_reg_dnn=0, l2_reg_cin=0,
seed=1024, dnn_dropout=0,dnn_activation='relu',
dnn_use_bn=False, task='binary')

2022-11-20 11:23:22.675695: W tensorflow/stream_executor/platform/default/dso_loader.cc:65] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib:/usr/local/cuda/lib64:/usr/local/nvidia/lib64:/usr/include/x86_64-linux-gnu
2022-11-20 11:23:22.675746: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-20 11:23:22.675783: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-11-20 11:23:22.676061: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the app

In [89]:
# compiling the model
import tensorflow as tf
model.compile(optimizer='adam',loss= 'mse', 
              metrics=['accuracy', tf.keras.metrics.Precision(),tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])

In [90]:
# training the model
history = model.fit(train_model_input, train['label'].values,
                    batch_size = 256, epochs = 20, verbose = 2, validation_split = 0.2, )

Epoch 1/20
2440/2440 - 55s - loss: 0.0859 - accuracy: 0.8830 - precision: 0.7291 - recall: 0.5931 - auc: 0.8970 - val_loss: 0.0746 - val_accuracy: 0.9032 - val_precision: 0.7791 - val_recall: 0.6933 - val_auc: 0.9340
Epoch 2/20
2440/2440 - 51s - loss: 0.0644 - accuracy: 0.9228 - precision: 0.8005 - recall: 0.7317 - auc: 0.9443 - val_loss: 0.0757 - val_accuracy: 0.9077 - val_precision: 0.8115 - val_recall: 0.7497 - val_auc: 0.9488
Epoch 3/20
2440/2440 - 50s - loss: 0.0468 - accuracy: 0.9504 - precision: 0.8281 - recall: 0.7726 - auc: 0.9547 - val_loss: 0.0810 - val_accuracy: 0.9033 - val_precision: 0.8369 - val_recall: 0.7867 - val_auc: 0.9576
Epoch 4/20
2440/2440 - 49s - loss: 0.0257 - accuracy: 0.9771 - precision: 0.8508 - recall: 0.8054 - auc: 0.9630 - val_loss: 0.0842 - val_accuracy: 0.9026 - val_precision: 0.8610 - val_recall: 0.8168 - val_auc: 0.9658
Epoch 5/20
2440/2440 - 49s - loss: 0.0238 - accuracy: 0.9782 - precision: 0.8701 - recall: 0.8268 - auc: 0.9684 - val_loss: 0.0844 -

In [91]:
#predicting
pred_ans_xdeep_test = model.predict(test_model_input, batch_size=256)

In [92]:
pred_ans_xdeep_test

array([[1.6844071e-08],
       [9.0317309e-01],
       [4.6040655e-05],
       ...,
       [3.3373708e-01],
       [3.7858665e-02],
       [9.6600127e-01]], dtype=float32)

In [93]:
#predicting
pred_ans_xdeep_train = model.predict(train_model_input, batch_size=256)

In [94]:
pred_ans_xdeep_train

array([[2.4282577e-05],
       [7.4285269e-04],
       [7.2456912e-09],
       ...,
       [4.4234097e-03],
       [5.9446096e-03],
       [8.8477999e-02]], dtype=float32)

In [95]:
test['pred_ctr'] = pred_ans_xdeep_test

In [96]:
train['pred_ctr'] = pred_ans_xdeep_train

In [97]:
ctr = pd.concat([train,test])
ctr # final data

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher,label,pred_ctr
271653,-0.158232,-0.268906,0.142092,0.492242,0.057918,-0.283251,-0.045025,0.186156,-0.314833,0.059551,...,2014.0,종교,기독교개신교,성경학습,복음서,레프 톨스토이홍익재,82247,25356,0,2.428258e-05
702435,-0.078569,0.022455,0.058227,0.138560,-0.319182,0.096117,0.332638,0.090844,-0.136288,0.145010,...,2006.0,외국어,어린이영어,유아영어,알파벳단어,편집부,237109,1298,0,7.428527e-04
638402,-0.115556,-0.199527,0.084906,0.433264,0.060257,-0.254176,-0.016829,0.076659,-0.238751,0.054459,...,2016.0,인문,언어학,한국어한국사한자능력시험,한국사능력검정시험,오창훈서울고시각,158735,11972,0,7.245691e-09
327652,0.056604,0.002474,0.317784,0.032473,0.156619,0.037441,-0.097863,0.197767,-0.479874,-0.096783,...,2010.0,자기계발,비즈니스능력계발,고전에서배우는리더십,고전에서배우는리더십,김우일영림카디널,58385,15958,0,1.517826e-02
303417,-0.285246,-0.057160,0.173872,0.314764,-0.079823,-0.082624,0.196562,0.095788,-0.442128,0.168964,...,2009.0,취업수험서,경제금융회계자격증,손해사정사,손해사정사,김주동대구대학교출판부,61686,5483,0,3.588360e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522839,0.014196,0.052331,0.398430,0.136520,-0.017091,-0.082762,0.269398,-0.029173,-0.136873,0.040368,...,2018.0,중고등참고서,중학교 학년별,중문제집,수학,최문섭 박선영 조햇봄 최희영 강레오 민승기 임양진 최현정비상교육,221667,10719,0,3.560553e-05
262377,0.124788,-0.138562,0.254888,0.072455,0.011741,0.007989,-0.008860,0.152515,-0.269417,0.008404,...,2010.0,정치사회,사회문제복지,사회복지,사회복지법,김세돈기한재,55138,4015,0,2.657907e-08
501237,-0.208546,0.092843,0.166569,0.283102,-0.190186,-0.212575,0.153015,0.108500,-0.023910,0.093547,...,2015.0,소설,한국소설,로맨스소설,로맨스소설,송주희황금가지,130062,25464,1,3.337371e-01
288032,-0.357206,0.049018,0.044488,0.089903,-0.184064,0.025280,0.339351,0.168402,-0.409263,-0.178799,...,2001.0,소설,한국소설,한국소설일반,한국소설일반,최용운문이당,223341,8574,1,3.785866e-02


In [98]:
ctr

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher,label,pred_ctr
271653,-0.158232,-0.268906,0.142092,0.492242,0.057918,-0.283251,-0.045025,0.186156,-0.314833,0.059551,...,2014.0,종교,기독교개신교,성경학습,복음서,레프 톨스토이홍익재,82247,25356,0,2.428258e-05
702435,-0.078569,0.022455,0.058227,0.138560,-0.319182,0.096117,0.332638,0.090844,-0.136288,0.145010,...,2006.0,외국어,어린이영어,유아영어,알파벳단어,편집부,237109,1298,0,7.428527e-04
638402,-0.115556,-0.199527,0.084906,0.433264,0.060257,-0.254176,-0.016829,0.076659,-0.238751,0.054459,...,2016.0,인문,언어학,한국어한국사한자능력시험,한국사능력검정시험,오창훈서울고시각,158735,11972,0,7.245691e-09
327652,0.056604,0.002474,0.317784,0.032473,0.156619,0.037441,-0.097863,0.197767,-0.479874,-0.096783,...,2010.0,자기계발,비즈니스능력계발,고전에서배우는리더십,고전에서배우는리더십,김우일영림카디널,58385,15958,0,1.517826e-02
303417,-0.285246,-0.057160,0.173872,0.314764,-0.079823,-0.082624,0.196562,0.095788,-0.442128,0.168964,...,2009.0,취업수험서,경제금융회계자격증,손해사정사,손해사정사,김주동대구대학교출판부,61686,5483,0,3.588360e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522839,0.014196,0.052331,0.398430,0.136520,-0.017091,-0.082762,0.269398,-0.029173,-0.136873,0.040368,...,2018.0,중고등참고서,중학교 학년별,중문제집,수학,최문섭 박선영 조햇봄 최희영 강레오 민승기 임양진 최현정비상교육,221667,10719,0,3.560553e-05
262377,0.124788,-0.138562,0.254888,0.072455,0.011741,0.007989,-0.008860,0.152515,-0.269417,0.008404,...,2010.0,정치사회,사회문제복지,사회복지,사회복지법,김세돈기한재,55138,4015,0,2.657907e-08
501237,-0.208546,0.092843,0.166569,0.283102,-0.190186,-0.212575,0.153015,0.108500,-0.023910,0.093547,...,2015.0,소설,한국소설,로맨스소설,로맨스소설,송주희황금가지,130062,25464,1,3.337371e-01
288032,-0.357206,0.049018,0.044488,0.089903,-0.184064,0.025280,0.339351,0.168402,-0.409263,-0.178799,...,2001.0,소설,한국소설,한국소설일반,한국소설일반,최용운문이당,223341,8574,1,3.785866e-02


In [99]:
ctr_lank = ctr.sort_values(by=ctr.columns[-1],ascending=False)

In [100]:
ctr_lank

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher,label,pred_ctr
475294,-0.190208,-0.161441,0.012509,0.289597,-0.131157,-0.200549,0.612289,-0.067332,-0.303410,-0.127841,...,2014.0,시에세이,한국시,현대시,현대시,이기선지식과감성,173489,19764,1,1.0
337308,-0.088391,-0.022854,0.111463,0.068271,-0.282465,0.090629,0.290843,0.182345,-0.199176,0.244138,...,2005.0,어린이초등,어린이만화,학습만화,한국사인물문화지리,유니트픽쳐담터미디어,165665,5427,1,1.0
485871,-0.183738,-0.216121,0.147259,0.203698,-0.189369,0.107465,0.358174,-0.085145,-0.179225,0.021183,...,2014.0,어린이초등,어린이문학,동화책,외국작가,크리스틴 베젤형설아이,231313,25161,1,1.0
879921,-0.202651,0.021184,0.045834,0.215969,-0.055504,-0.339933,0.033845,0.144874,-0.126301,0.077293,...,2022.0,어린이초등,어린이문학,동화책,한국작가,류미정노란돼지,85867,4745,1,1.0
585352,-0.086916,-0.101678,0.147700,0.128245,-0.261176,0.130868,0.433380,-0.085582,-0.161184,0.078812,...,2018.0,시에세이,나라별 에세이,한국에세이,한국에세이,강혜정지원출판사,35737,19890,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212552,-0.218470,-0.091464,0.283737,0.144930,-0.064031,-0.044173,0.077355,0.012684,-0.201593,-0.078697,...,2018.0,예술대중문화,미술,교양미술,미술이야기,이주은아트북스,183917,14640,0,0.0
216876,-0.027891,0.008059,-0.011197,0.017317,-0.001355,0.036775,0.053332,-0.031659,-0.032541,-0.000117,...,2013.0,예술대중문화,패션의류,패션,패션도식드레이핑,양경희에코모다,148822,15438,0,0.0
762805,-0.077507,-0.040993,0.184186,0.294730,-0.181184,-0.387956,0.119556,-0.362964,-0.461168,-0.099836,...,2021.0,어린이초등,어린이교양,예술취미,취미실용,꿈소담이 편집부소담주니어,66700,12648,0,0.0
637948,-0.053629,0.051997,0.238697,0.141239,-0.142955,0.022341,0.291499,0.185885,-0.216344,0.200446,...,2020.0,정치사회,정부간행물,정부간행물,정부간행물,국세청국세청,43172,3407,0,0.0


In [101]:
lanktop = ctr_lank[:30]

In [102]:
lanktop

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher,label,pred_ctr
475294,-0.190208,-0.161441,0.012509,0.289597,-0.131157,-0.200549,0.612289,-0.067332,-0.303410,-0.127841,...,2014.0,시에세이,한국시,현대시,현대시,이기선지식과감성,173489,19764,1,1.0
337308,-0.088391,-0.022854,0.111463,0.068271,-0.282465,0.090629,0.290843,0.182345,-0.199176,0.244138,...,2005.0,어린이초등,어린이만화,학습만화,한국사인물문화지리,유니트픽쳐담터미디어,165665,5427,1,1.0
485871,-0.183738,-0.216121,0.147259,0.203698,-0.189369,0.107465,0.358174,-0.085145,-0.179225,0.021183,...,2014.0,어린이초등,어린이문학,동화책,외국작가,크리스틴 베젤형설아이,231313,25161,1,1.0
879921,-0.202651,0.021184,0.045834,0.215969,-0.055504,-0.339933,0.033845,0.144874,-0.126301,0.077293,...,2022.0,어린이초등,어린이문학,동화책,한국작가,류미정노란돼지,85867,4745,1,1.0
585352,-0.086916,-0.101678,0.147700,0.128245,-0.261176,0.130868,0.433380,-0.085582,-0.161184,0.078812,...,2018.0,시에세이,나라별 에세이,한국에세이,한국에세이,강혜정지원출판사,35737,19890,1,1.0
971339,-0.342829,0.062408,0.060904,0.356829,0.063134,0.138457,0.482264,0.064869,-0.086900,0.063252,...,2008.0,소설,한국소설,로맨스소설,로맨스소설,이민경서울문화사,175779,12002,1,1.0
267663,-0.287777,-0.092155,0.259168,0.249529,-0.226300,-0.032193,0.102352,-0.099717,-0.107833,-0.075492,...,2009.0,시에세이,나라별 에세이,한국에세이,한국에세이,임춘식동아일보사,190880,6479,1,1.0
838446,0.021045,0.085137,0.355626,0.166619,-0.074656,-0.107552,0.316736,-0.043358,-0.124448,0.108348,...,2014.0,유아세,유아그림책,외국그림책,외국그림책,모 윌렘스살림어린이,94908,11282,1,1.0
905154,-0.236716,0.052403,0.434504,0.207576,-0.183956,0.038031,0.170086,0.078087,-0.304169,0.008409,...,2022.0,어린이초등,어린이만화,만화애니메이션,만화일반,예씨 원작 안경순서울문화사,155710,12002,1,1.0
594734,-0.087127,0.025227,0.224782,0.288928,-0.277938,-0.234751,0.363008,-0.024849,-0.465030,-0.147361,...,2020.0,어린이초등,어린이문학,동화책,외국작가,오언 콜퍼위니더북,157698,17213,1,1.0


In [ ]:
lanktop

In [109]:
like_cn3

['디스크척추', '요통', '외과']

In [ ]:
## 카테고리제한걸경우

In [110]:
userlike = ctr_lank['Cn_3'].isin(like_cn3)
like = ctr_lank[userlike]

# userlikecn3필터링거친 추천결과

In [111]:
like

,catev_0,catev_1,catev_2,catev_3,catev_4,catev_5,catev_6,catev_7,catev_8,catev_9,...,date,Cn_1,Cn_2,Cn_3,Cn_4,author_publisher,author,publisher,label,pred_ctr
850861,-0.240171,-0.213571,-0.172365,0.316750,-0.086984,-0.033023,0.213558,0.006922,-0.015595,-0.010826,...,2021.0,기술공학,의학,외과,근육근골격계질환,무라오카 이사오 감수성안당,95984,12325,1,1.000000
155824,-0.032986,-0.011480,0.338695,0.067835,-0.108876,-0.061934,0.275289,-0.084773,-0.066646,0.125161,...,2013.0,기술공학,의학,외과,근육근골격계질환,유홍종양서각,168260,14961,1,1.000000
850220,-0.073348,-0.091289,0.153150,0.018290,-0.056577,-0.137595,0.542312,-0.086129,-0.282126,-0.159335,...,2009.0,기술공학,의학,외과,근육근골격계질환,범한서적,24631,9668,1,1.000000
259953,-0.200192,0.103716,0.125293,-0.046410,-0.310331,-0.117015,0.073700,-0.051613,-0.253250,-0.096825,...,2019.0,기술공학,의학,외과,물리치료학,나카이 코지대학서림,68222,5661,1,1.000000
506068,0.092858,-0.121410,0.277534,0.078927,0.082784,0.002683,-0.036485,0.136031,-0.358711,-0.035545,...,2018.0,기술공학,의학,외과,성형외과,대한미용성형외과학회군자출판사,75408,3497,1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267488,-0.303419,-0.006979,0.161122,0.136333,0.089628,-0.235733,-0.000694,0.085887,-0.344932,0.119032,...,2008.0,기술공학,의학,외과,외과일반,대한췌담도학회군자출판사,75780,3497,1,0.000764
129672,-0.210496,-0.031097,0.210029,-0.033451,-0.213816,-0.140596,0.041910,0.093873,-0.249039,-0.043452,...,2019.0,기술공학,의학,외과,전기수광선치료학,김용식서울대학교출판문화원,58107,11989,1,0.000664
599607,-0.089737,0.235024,0.149370,0.028674,-0.330808,0.006014,0.272737,0.024954,-0.058129,0.031946,...,2014.0,기술공학,의학,외과,성형외과,김연정가담플러스,56765,2270,1,0.000643
740856,-0.177391,0.174954,0.119593,0.155318,0.006570,-0.360535,-0.049241,0.278223,-0.017547,0.055077,...,2001.0,기술공학,의학,외과,물리치료학,장택환동남기획,194912,6364,1,0.000643


# userlikecn3필터링거치지않은 추천결과

In [103]:
print("<<{}님의 책추천 결과입니다.>>".format(customer_idx))
print(lanktop['title'])

<<5151님의 책추천 결과입니다.>>
475294                                            파리, 날아가다
337308                                 제트 레이서 3(한반도 대탐험 편)
485871    유령 공주의 숨바꼭질(디에고 벨라스케스의)(상상 미술관 9)(양장본 HardCover)
879921                                           엄마는 트롯 가수
585352                           너만 이렇게 사냐고? 아니, 나도 이렇게 살아
971339                                               첫키스 1
267663                                           소통과 창조(5)
838446                              모 윌렘스의 비둘기 시리즈 세트(전5권)
905154               예씨 금손 똥손 2: 이상한 나라의 예씨(양장본 HardCover)
594734                                     하늘을 나는 발명왕 마리엘라
90315                                    오싹오싹 공포 체험 스쿨버스 1
57932                   수학도둑 54: 창의편(코믹 메이플스토리)(코믹 메이플스토리)
270535                                   작은 시 한수로 사랑한다는 것은
349257                            정의(성화된 삶을 위한 말씀동행 시리즈 3)
153856                         2020 크레용하우스 필독서 세트(유아)(전5권)
233950                              웬만해선 그녀의 컴플레인을 막을 수 없다
293586                            